### XNAT DB Connection and OpenVINO model server Inference Example.
This is a very basic sample example which does the following:
1. Start OpenVINO model Server
1. Connect to  XNAT DB
2. Download a sample DICOM Image form XNAT DB
3. Read the DICOM image using pydicom
4. Send the data to OpenVINO model server for inference.

### Start OpenVINO model server. In the terminal, run the following:
```
docker run --rm \
-v /home/bduser/ravi/xnat/ovms/models/unet/FP32:/opt/ml/unet/1 \
-p 9001:9001 -p 8001:8001 ie-serving-py \
/ie-serving-py/start_server.sh ie_serving model \
--model_path /opt/ml/unet --model_name unet \
--port 9001 --rest_port 8001 
```

#### Sample output in the terminal
```
2020-03-31 20:14:37,144 - ie_serving.main - INFO - Log level set: INFO
2020-03-31 20:14:37,144 - ie_serving.models.model - INFO - Server start loading model: unet
2020-03-31 20:14:37,146 - ie_serving.models.model - INFO - Creating inference engine object for version: 1
2020-03-31 20:14:37,314 - ie_serving.models.ir_engine - INFO - Matched keys for model: {'inputs': {'MRImages': 'MRImages'}, 'outputs': {'PredictionMask/Sigmoid': 'PredictionMask/Sigmoid'}}
2020-03-31 20:14:37,314 - ie_serving.models.model - INFO - List of available versions for unet model: [1]
2020-03-31 20:14:37,314 - ie_serving.models.model - INFO - Default version for unet model is 1
2020-03-31 20:14:37,321 - ie_serving.server.start - INFO - REST server listens on port 8001 and will be serving models: ['unet']
2020-03-31 20:14:37,326 - ie_serving.server.start - INFO - gRPC server listens on port 9001 and will be serving models: ['unet']
```

### Note: You can run in the `detached` mode using `-d` if you are using Jupyter Notebook.

In [16]:
%%sh
docker run -d --rm \
-v /home/bduser/ravi/xnat/ovms/models/unet/FP32:/opt/ml/unet/1 \
-p 9001:9001 -p 8001:8001 ie-serving-py \
/ie-serving-py/start_server.sh ie_serving model \
--model_path /opt/ml/unet --model_name unet \
--port 9001 --rest_port 8001

7770104b54a724d3546fcc33750fe473d4ec6d342f49a3c4610e0c1e26d143ca


In [4]:
# Connect to XNAT database
import xnat
connection = xnat.connect('https://central.xnat.org', user="rpanchum", password="ravi123")

print("Connection successful ! ")

# Get a sample DICOM image
project = 'Sample_DICOM'
subject = 'dcmtest1'
session = 0
scan = 0
resource = 'DICOM'
file_obj = connection.projects[project].subjects[subject].experiments[session].scans[scan].resources[resource].files[0]

print("Read a DICOM scan successfully...")



Connection successful ! 
Read a DICOM scan successfully...


In [5]:
# Read the DICOM image 
import pydicom
with file_obj.open() as fin:
    dataset = pydicom.dcmread(fin)

# Print DICOM Data Info
print()
print("Filename.........:", file_obj.fulluri)
print("Storage type.....:", dataset.SOPClassUID)
print()

pat_name = dataset.PatientName
display_name = pat_name.family_name + ", " + pat_name.given_name
print("Patient's name...:", display_name)
print("Patient id.......:", dataset.PatientID)
print("Modality.........:", dataset.Modality)
print("Study Date.......:", dataset.StudyDate)

if 'PixelData' in dataset:
    rows = int(dataset.Rows)
    cols = int(dataset.Columns)
    print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
        rows=rows, cols=cols, size=len(dataset.PixelData)))
    if 'PixelSpacing' in dataset:
        print("Pixel spacing....:", dataset.PixelSpacing)

    dicom_image_arr = dataset.pixel_array


Filename.........: /data/projects/Sample_DICOM/subjects/CENTRAL_S01894/experiments/CENTRAL_E03907/scans/2/resources/123221318/files/dcmtest1.MR.Sample_DICOM.2.1.20010108.120022.1azj8tu.dcm
Storage type.....: 1.2.840.10008.5.1.4.1.1.4

Patient's name...: dcmtest1, 
Patient id.......: dcmtest1_MR1
Modality.........: MR
Study Date.......: 20010108
Image size.......: 512 x 512, 524288 bytes
Pixel spacing....: [0.390625, 0.390625]


#### Send Inference request like below

In [6]:
import numpy as np
import json
import requests
import time

# Create a dummy data for UNet, as the dicom image read above does not have 4 channels.
input_data = np.random.randint(0, 255, size=(1, 4, 160,160))

data_obj = {'inputs':  input_data.tolist()}
data_json = json.dumps(data_obj)
#print(data_json)

start_time = time.time()
request_url = "http://localhost:8001/v1/models/unet:predict"
result = requests.post(request_url, data=data_json)
result_dict = json.loads(result.text)
delta = (time.time() - start_time)

print("Request URL:", request_url)
print("Input shape:", input_data.shape)
print("Output shape:", np.asarray(result_dict['outputs']).shape)
print("Completed Inference with one sample in {:.3f} sec,".format(delta))

Request URL: http://localhost:8001/v1/models/unet:predict
Input shape: (1, 4, 160, 160)
Output shape: (1, 1, 160, 160)
Completed Inference with one sample in 0.066 sec,
